# Étude de Cas Complète : Analyse de Données et Modélisation

Dans ce notebook, nous allons réaliser une étude de cas complète sur un dataset de diamants.
Nous suivrons les étapes typiques d'un projet de Machine Learning :

1. Exploration et nettoyage des données
2. Feature engineering et préparation
3. Modélisation et évaluation
4. Optimisation des modèles
5. Analyse des résultats

In [ ]:
# Imports nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

## 1. Chargement et Exploration des Données

In [ ]:
# Chargement du dataset
try:
    df = sns.load_dataset("diamonds")
    dataset_name = "Diamonds"
except:
    from sklearn.datasets import fetch_california_housing
    data = fetch_california_housing(as_frame=True)
    df = data.frame.copy()
    dataset_name = "California Housing"

print(f"Dataset utilisé : {dataset_name}")
print("\nAperçu des données :")
display(df.head())
print("\nInformations sur le dataset :")
df.info()

In [ ]:
# Analyse statistique descriptive
print("Statistiques descriptives :")
display(df.describe())

# Visualisation de la distribution des variables numériques
plt.figure(figsize=(15, 10))
df.select_dtypes(include=[np.number]).hist(bins=30)
plt.tight_layout()
plt.show()

## 2. Nettoyage et Préparation des Données

In [ ]:
# Nettoyage des données
print("Valeurs manquantes avant nettoyage :")
display(df.isnull().sum())

df = df.dropna()
print("\nValeurs manquantes après nettoyage :")
display(df.isnull().sum())

In [ ]:
# Préparation des features
target = df.columns[-1]
X = df.drop(columns=[target])
y = df[target]

# Encodage des variables catégorielles si présentes
X = pd.get_dummies(X, drop_first=True)

print("Features après encodage :")
print(f"Nombre de features : {X.shape[1]}")
print("Liste des features :", X.columns.tolist())

## 3. Split Train/Test et Standardisation

In [ ]:
# Split des données
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardisation
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Dimensions des données :")
print(f"X_train : {X_train_scaled.shape}")
print(f"X_test : {X_test_scaled.shape}")
print(f"y_train : {y_train.shape}")
print(f"y_test : {y_test.shape}")

## 4. Modélisation Initiale

In [ ]:
# Définition des modèles
models = {
    "Ridge": Ridge(),
    "RandomForest": RandomForestRegressor(random_state=42)
}

# Évaluation initiale
results = []
print("Évaluation initiale des modèles :")
for name, model in models.items():
    # Cross-validation
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='r2')
    
    # Entraînement sur l'ensemble complet
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        'Modèle': name,
        'R² CV Moyen': cv_scores.mean(),
        'R² CV Std': cv_scores.std(),
        'RMSE Test': rmse,
        'R² Test': r2
    })

results_df = pd.DataFrame(results)
display(results_df)

## 5. Optimisation des Modèles

In [ ]:
# Définition des grilles de paramètres
param_grids = {
    "Ridge": {
        "alpha": [0.1, 1.0, 10.0],
        "solver": ['auto', 'svd', 'cholesky']
    },
    "RandomForest": {
        "n_estimators": [100, 200],
        "max_depth": [None, 10, 20],
        "min_samples_split": [2, 5]
    }
}

# Optimisation et évaluation
best_models = {}
final_results = []

for name, model in models.items():
    print(f"\nOptimisation de {name}...")
    grid = GridSearchCV(model, param_grids[name], cv=5, scoring='r2', n_jobs=-1)
    grid.fit(X_train_scaled, y_train)
    
    best_models[name] = grid.best_estimator_
    y_pred = grid.best_estimator_.predict(X_test_scaled)
    
    final_results.append({
        'Modèle': name,
        'Meilleurs Paramètres': grid.best_params_,
        'R² CV': grid.best_score_,
        'RMSE Test': mean_squared_error(y_test, y_pred, squared=False),
        'R² Test': r2_score(y_test, y_pred)
    })
    
final_results_df = pd.DataFrame(final_results)
display(final_results_df)

## 6. Visualisation des Résultats

In [ ]:
# Comparaison des prédictions
plt.figure(figsize=(15, 5))
for idx, (name, model) in enumerate(best_models.items(), 1):
    plt.subplot(1, 2, idx)
    y_pred = model.predict(X_test_scaled)
    
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
    plt.xlabel('Valeurs Réelles')
    plt.ylabel('Prédictions')
    plt.title(f'{name}')

plt.tight_layout()
plt.show()

## 7. Conclusion et Perspectives

Dans cette étude de cas, nous avons :
1. Exploré et nettoyé les données
2. Préparé les features avec encodage et standardisation
3. Évalué deux modèles différents (Ridge et Random Forest)
4. Optimisé les hyperparamètres
5. Comparé les performances finales

Pour le projet final, il est recommandé de :
- Choisir un dataset personnel sur Kaggle ou OpenML
- Définir clairement l'objectif (classification ou régression)
- Organiser le code comme un vrai projet avec :
  - `src/` pour le code source
  - `notebooks/` pour l'exploration
  - `scripts/` pour les utilitaires
  - Un README détaillé